In [1]:
# Import des bibliothèques et modules nécessaires
import warnings
warnings.filterwarnings("ignore")
import pandas as pd 
import plotly.express as px
import dash 
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash

In [2]:
# Lecture des données par continent
# Afrique
data_Africa = pd.read_csv('https://github.com/Ayoub-Briguiche/Dash_plotly/raw/main/Emissions_Totals_E_Africa.csv', encoding='latin-1')
# Amériques
data_Americas = pd.read_csv('https://github.com/Ayoub-Briguiche/Dash_plotly/raw/main/Emissions_Totals_E_Americas.csv', encoding='latin-1')
# Asie
data_Asia = pd.read_csv('https://github.com/Ayoub-Briguiche/Dash_plotly/raw/main/Emissions_Totals_E_Asia.csv', encoding='latin-1')
# Europe
data_Europe = pd.read_csv('https://github.com/Ayoub-Briguiche/Dash_plotly/raw/main/Emissions_Totals_E_Europe.csv', encoding='latin-1')
# Océanie
data_Oceania = pd.read_csv('https://github.com/Ayoub-Briguiche/Dash_plotly/raw/main/Emissions_Totals_E_Oceania.csv', encoding='latin-1')

In [3]:
# Ajout de la colonne 'continent' pour chaque DataFrame
data_Africa['continent'] = 'Africa'
data_Americas['continent'] = 'Americas'
data_Asia['continent'] = 'Asia'
data_Europe['continent'] = 'Europe'
data_Oceania['continent'] = 'Oceania'

# Fusion de tous les DataFrames en un seul
df = pd.concat([data_Europe, data_Africa, data_Americas, data_Asia, data_Oceania], ignore_index=True)
df.head()

,Area Code,Area Code (M49),Area,Item Code,Item,Element Code,Element,Source Code,Source,Unit,...,Y2021,Y2021F,Y2021N,Y2030,Y2030F,Y2030N,Y2050,Y2050F,Y2050N,continent
0,3,'008,Albania,5064,Crop Residues,7234,Direct emissions (N2O),3050,FAO TIER 1,kt,...,0.1179,E,NaN,0.1132,F,NaN,0.1159,F,NaN,Europe
1,3,'008,Albania,5064,Crop Residues,7236,Indirect emissions (N2O),3050,FAO TIER 1,kt,...,0.0265,E,NaN,0.0255,F,NaN,0.0261,F,NaN,Europe
2,3,'008,Albania,5064,Crop Residues,7230,Emissions (N2O),3050,FAO TIER 1,kt,...,0.1444,E,NaN,0.1387,F,NaN,0.1420,F,NaN,Europe
3,3,'008,Albania,5064,Crop Residues,724313,Emissions (CO2eq) from N2O (AR5),3050,FAO TIER 1,kt,...,38.2660,E,NaN,36.7555,F,NaN,37.6300,F,NaN,Europe
4,3,'008,Albania,5064,Crop Residues,723113,Emissions (CO2eq) (AR5),3050,FAO TIER 1,kt,...,38.2660,E,NaN,36.7555,F,NaN,37.6300,F,NaN,Europe


In [4]:
# Lecture du fichier word.csv pour obtenir les codes ISO3 des régions
word = pd.read_csv('https://raw.githubusercontent.com/Ayoub-Briguiche/Dash_plotly/main/word.csv', encoding='latin-1')
Area_Code = word[['Area Code (ISO3)','Area']]
Area_Code.head()

,Area Code (ISO3),Area
0,AFG,Afghanistan
1,ALB,Albania
2,DZA,Algeria
3,AGO,Angola
4,ATG,Antigua and Barbuda


In [5]:
# Fusion des données avec les codes ISO3
data = pd.merge(df, Area_Code, on='Area', how='left')
data.head()

,Area Code,Area Code (M49),Area,Item Code,Item,Element Code,Element,Source Code,Source,Unit,...,Y2021F,Y2021N,Y2030,Y2030F,Y2030N,Y2050,Y2050F,Y2050N,continent,Area Code (ISO3)
0,3,'008,Albania,5064,Crop Residues,7234,Direct emissions (N2O),3050,FAO TIER 1,kt,...,E,NaN,0.1132,F,NaN,0.1159,F,NaN,Europe,ALB
1,3,'008,Albania,5064,Crop Residues,7236,Indirect emissions (N2O),3050,FAO TIER 1,kt,...,E,NaN,0.0255,F,NaN,0.0261,F,NaN,Europe,ALB
2,3,'008,Albania,5064,Crop Residues,7230,Emissions (N2O),3050,FAO TIER 1,kt,...,E,NaN,0.1387,F,NaN,0.1420,F,NaN,Europe,ALB
3,3,'008,Albania,5064,Crop Residues,724313,Emissions (CO2eq) from N2O (AR5),3050,FAO TIER 1,kt,...,E,NaN,36.7555,F,NaN,37.6300,F,NaN,Europe,ALB
4,3,'008,Albania,5064,Crop Residues,723113,Emissions (CO2eq) (AR5),3050,FAO TIER 1,kt,...,E,NaN,36.7555,F,NaN,37.6300,F,NaN,Europe,ALB


In [6]:
# Suppression des colonnes inutiles
data.drop(columns=['Area Code','Area Code (M49)','Item Code','Element Code','Source Code','Y2030','Y2050','Unit'], inplace=True)

In [7]:
# Suppression des colonnes se terminant par 'NF' ou 'N'
colonnes_terminant_par_NF = [colonne for colonne in data.columns if colonne.endswith('F') or colonne.endswith('N')]
data.drop(columns=colonnes_terminant_par_NF, inplace=True)

In [8]:
# Changement des noms des colonnes ('Yannée' --> 'année')
year = []
for i in range(1961, 2022):
    year.append(i)

Yyear = data.columns.tolist()[4:]

columns = {}
for x, y in zip(Yyear, year):
    columns[x] = y

data = data.rename(columns=columns)
data.head()

,Area,Item,Element,Source,1961,1962,1963,1964,1965,1966,...,2014,2015,2016,2017,2018,2019,2020,2021,continent,Area Code (ISO3)
0,Albania,Crop Residues,Direct emissions (N2O),FAO TIER 1,0.0721,0.0760,0.0666,0.0816,0.0765,0.0959,...,0.1228,0.1219,0.1222,0.1224,0.1174,0.1151,0.1173,0.1179,Europe,ALB
1,Albania,Crop Residues,Indirect emissions (N2O),FAO TIER 1,0.0162,0.0171,0.0150,0.0184,0.0172,0.0216,...,0.0276,0.0274,0.0275,0.0275,0.0264,0.0259,0.0264,0.0265,Europe,ALB
2,Albania,Crop Residues,Emissions (N2O),FAO TIER 1,0.0883,0.0931,0.0816,0.1000,0.0937,0.1174,...,0.1504,0.1494,0.1497,0.1499,0.1438,0.1410,0.1437,0.1444,Europe,ALB
3,Albania,Crop Residues,Emissions (CO2eq) from N2O (AR5),FAO TIER 1,23.3995,24.6715,21.6240,26.5000,24.8305,31.1110,...,39.8560,39.5910,39.6705,39.7235,38.1070,37.3650,38.0805,38.2660,Europe,ALB
4,Albania,Crop Residues,Emissions (CO2eq) (AR5),FAO TIER 1,23.3995,24.6715,21.6240,26.5000,24.8305,31.1110,...,39.8560,39.5910,39.6705,39.7235,38.1070,37.3650,38.0805,38.2660,Europe,ALB


In [9]:
# Transformation de la table en une seule dimension
df_transforme = pd.melt(data, id_vars=['continent','Area Code (ISO3)','Area','Item', 'Element',"Source"], var_name='Year', value_name='Emission_value')

# Suppression des lignes avec des valeurs manquantes
df_transforme.dropna(axis=0, inplace=True)
df_transforme.head()

,continent,Area Code (ISO3),Area,Item,Element,Source,Year,Emission_value
0,Europe,ALB,Albania,Crop Residues,Direct emissions (N2O),FAO TIER 1,1961,0.0721
1,Europe,ALB,Albania,Crop Residues,Indirect emissions (N2O),FAO TIER 1,1961,0.0162
2,Europe,ALB,Albania,Crop Residues,Emissions (N2O),FAO TIER 1,1961,0.0883
3,Europe,ALB,Albania,Crop Residues,Emissions (CO2eq) from N2O (AR5),FAO TIER 1,1961,23.3995
4,Europe,ALB,Albania,Crop Residues,Emissions (CO2eq) (AR5),FAO TIER 1,1961,23.3995


In [10]:
# Définition des options pour les filtres
Element_f = [{'label': el, 'value': el} for el in df_transforme['Element'].unique()]
Item_f = [{'label': item, 'value': item} for item in df_transforme['Item'].unique()]
source_f = [{'label': source, 'value': source} for source in df_transforme['Source'].unique()]
agg = [{'label': 'Sum', 'value': 'sum'}, {'label': 'Average', 'value': 'mean'}]

# Liste des catégories d'émissions agricoles
liste_agro = ['Food Processing','Food Transport','Food Retail','Food Packaging','Agrifood Systems Waste Disposal',
              'Pesticides Manufacturi','Food Household Consumption','Synthetic Fertilizers','On-farm energy us',
              'Fires in humid tropical forests', 'Forest fires','Net Forest conversion','Emissions from livestock',
              'Savanna fires','Fires in organic soils','Drained organic soils (CO2)','Fertilizers Manufacturing', 
              'Emissions from crops']

# Fonctions utilitaires
def make_break(num_breaks):
    # Crée une liste de sauts de ligne
    br_list = [html.Br()] * num_breaks
    return br_list

def add_logo():
    # Ajoute un logo 
    corp_logo = html.Img(src='https://i.ibb.co/ctPZxCm/qrcode-99577810-d173fa986f46821fd3cd42de81f7f78e-1.png',
                         style={'width': 80, 'color': '#','margin': 'auto'})
    return corp_logo

def add_logo2():
    # Ajoute un autre logo 
    corp_logo = html.Img(src='https://i.ibb.co/3Sqm77T/7207396.png',
                         style={'width': 400, 'left': 400,'align-items': 'center','margin-left':'10px',
                                'text-align': 'center','border-radius': '20px','display': 'flex', 
                                'flex-direction': 'row'})
    return corp_logo

def add_logo3():
    # Ajoute un troisième logo 
    corp_logo = html.Img(src='https://i.ibb.co/QvwxPmH/Nouveau-projet.png',
                         style={'width': 200, 'left': 200,
                                'text-align': 'center','border-radius': '20px'})
    return corp_logo

def style_c():
    # Définit un style de mise en forme pour les éléments HTML
    layout_style = {'display': 'inline-block', 'margin': '0 auto', 'padding': '20px'}
    return layout_style


In [13]:
# Définition de l'application Dash
app =dash.Dash(__name__)
# Mise en page de l'application
app.layout = html.Div([
    html.Div(
        children=[
            *make_break(1),
            html.Div(children=[
                html.Div(children=[
                    html.H5('Elements',style={'text-align': 'center', 'margin': 'auto'}),
                    dcc.Dropdown(id='Element_filtre',options =Element_f,placeholder="Select Element...",
                                 value=Element_f[0]['value'], 
                                 style={'height': '80%','align-items': 'center','width': 300, 'text-align': 'center',
                                        'color': '#','border-radius': '10px','font-size': '14px',  'justify-content': 'center',
                                        'margin': 'auto','background-color':'#f9f9f8'} )],
                         style={'display': 'flex', 'flex-direction': 'column','align-items': 'center','margin': 'auto'}
                        ),
                
                html.Div(children=[
                    html.H5('Items',style={'text-align': 'center','margin': 'auto'}),
                    dcc.Dropdown(id='Item_filtre',options =Item_f ,placeholder="Select Item...",
                                 value=Item_f[0]['value'], 
                                 style={'height': '80%','align-items': 'center','width': 300, 'text-align': 'center',
                                        'color': '#','border-radius': '10px','font-size': '14px',  'justify-content': 'center',
                                        'margin': 'auto','background-color':'#f9f9f8'} )],
                         style={'display': 'flex', 'flex-direction': 'column','align-items': 'center','margin': 'auto'}
                        ),
                html.Div(children=[
                    html.H5('Source',style={'text-align': 'center', 'margin': 'auto'}),
                    dcc.Dropdown(id='Source_filtre',options = source_f, placeholder="Select a Area...",
                                 value=source_f[0]['value'], 
                                 style={'height': '80%','width': 300,'font-size': '14px',  'justify-content': 'center',
                                        'border-radius': '10px','background-color':'#f9f9f8'} )],
                         style={'display': 'flex', 'flex-direction': 'column','margin': 'auto'}
                        ),
                
            ],
                     style={'height': '10%','width': "95%",'align-items': 'center',
                            'text-align': 'center','margin': 'auto','border-radius': '20px',
                            'box-shadow': '0px 0px 2px rgba(0, 0, 0, 0.5)','display': 'flex', 
                            'flex-direction': 'row','background-color':'#f9f9f8'}),
            
            *make_break(1),
            html.Div(children=[
                html.Div(
                    html.Div(children=[
                        dcc.Graph(id='carte')],style={'width': '95%'}
                            ),
                    style={'height': 500,'width': "70%",'align-items': 'center',
                           'text-align': 'center','border-radius': '20px',
                           'box-shadow': '0px 0px 2px rgba(0, 0, 0, 0.5)','display': 'flex', 
                           'flex-direction': 'row','background-color':'#f9f9f8'}
                ),
                
                html.Div(children=[
                    html.Div(children=[
                        html.H5('Total Emissions :'),
                        html.Div(id='somme',style={'font-size': '30px'} )],
                             style={'height': 120,'margin-left':'10px','text-align': 'center','border-radius': '20px',
                                    'box-shadow': '0px 0px 2px rgba(0, 0, 0, 0.5)','display': 'flex', 
                                    'flex-direction': 'column','background-color':'#f9f9f8'}
                            ),
                    *make_break(1),
                    html.Div(
                        add_logo2(),
                    )
                ],
                         style={'display': 'flex', 'flex-direction': 'column','width': "30%"}
                        )
            ],
                     style={'display': 'flex', 'flex-direction': 'row','margin': 'auto','width':"95%"}),
            *make_break(1),
            html.Div(children=[
                html.Div(
                    html.Div(children=[
                        dcc.Graph(id='Area_evolution')],style={'width': '95%','margin': '15px'}
                            ),
                    style={'height': 400,'width': "62%",'align-items': 'center','text-align': 'center','border-radius': '20px',
                           'box-shadow': '0px 0px 2px rgba(0, 0, 0, 0.5)','display': 'flex', 
                           'flex-direction': 'row','background-color':'#f9f9f8'}
                ),
                *make_break(1), 
                html.Div(
                    html.Div(children=[
                        dcc.Graph(id='continent_bar')],style={'width': '95%','margin': '25px'}
                            ),
                    style={'height': 400,'width': "38%",'align-items': 'center','margin-left':'10px',
                           'text-align': 'center','border-radius': '20px',
                           'box-shadow': '0px 0px 2px rgba(0, 0, 0, 0.5)','display': 'flex', 
                           'flex-direction': 'row','background-color':'#f9f9f8'}
                )
            ],
                     style={'display': 'flex', 'flex-direction': 'row','margin': 'auto','width':"95%"}),
            *make_break(1),
            html.Div(children=[
                html.Div(
                    html.Div(children=[
                        dcc.Graph(id='Area_con')],style={'width': "95%",'margin-left': '30px'}
                            ),
                    style={'height': 500,'width': "50%",'align-items': 'center','margin-left':'10px',
                           'text-align': 'center','border-radius': '20px',
                           'box-shadow': '0px 0px 2px rgba(0, 0, 0, 0.5)','display': 'flex', 
                           'flex-direction': 'row','background-color':'#f9f9f8'}
                ),
                *make_break(1),
                html.Div(
                    html.Div(children=[
                        dcc.Graph(id='xxxxx')],style={'width': "95%",'margin-left': '30px','top':0}
                            ),
                    style={'height': 500,'width': "50%",'align-items': 'center','margin-left':'10px',
                           'text-align': 'center','border-radius': '20px',
                           'box-shadow': '0px 0px 2px rgba(0, 0, 0, 0.5)','display': 'flex', 
                           'flex-direction': 'row','background-color':'#f9f9f8'}
                ),
            ],
                     style={'display': 'flex', 'flex-direction': 'row','margin': 'auto','width':"95%"})
        ],
        style={'position': 'absolute','top': 0,'right': "0px",'left': '15%','bottom': 0,'background_attachement':'fixed',
               'text-align': 'center','color': '#','position': 'fixed',
               'background-color':'#edefeb', 'overflow': 'auto'}),
    html.Div(
        children=[
            *make_break(2),
            html.H2('Environmental Analysis Dashboard',
                    style={'margin': '5px','color':'#edefeb'}),
            *make_break(1),
            html.H5('Through this dashboard, our objective is to delve into the intricate web of agricultural and environmental factors that wield significant influence over greenhouse gas emissions. Our aim is to meticulously dissect each type of gas emission and scrutinize every emission source. By doing so, we aspire to gain a comprehensive understanding of their individual and collective contributions to the broader landscape of greenhouse gas emissions.',
                    style={'margin': '10px','text-align': 'center'}),
            *make_break(1),
            add_logo3(),   
            *make_break(5),   
            html.Div(children=[
                html.Div(
                    add_logo(),
                    style={'height': 80,'width': 80,'align-items': 'center','margin-left': '10px','text-align': 'center',
                           'display': 'flex'}
                ),
                html.Div(children=[
                    html.Div(
                        children=[
                            html.Span('Ayoub Briguiche', style={'display': 'block','font-size': '16px'}),
                            *make_break(1),
                            html.A('briguicheayoub@gmail.com', href='mailto:briguicheayoub@gmail.com', style={'display': 'block','font-size': '12px'})
                        ],
                        style={'flex-direction': 'column','display': 'flex','text-align': 'left','margin-top': "12px"}
                    )
                ],
                         style={'height': 80,'width': 160}
                        )
            ],
                     style={'flex-direction': 'row','display': 'flex'}
                    ),
            *make_break(2),    
        ],
        style={'position': 'absolute','top': 0,'right': '85%','left': 0,'bottom': 0,'text-align': 'center','color': '#',
               'margin': '0px','background-color': '#4b9072','background_attachement': 'fixed'}
    ) ,
])
##################################################################
# Callback pour la somme
@app.callback(
    Output(component_id='somme', component_property='children'),
    [Input(component_id='Element_filtre', component_property='value'),
     Input(component_id='Item_filtre', component_property='value'),
     Input(component_id='Source_filtre', component_property='value')]
)
def update_plot(Element_search, Item_search, Source_search):
    # Copie du dataframe
    df = df_transforme.copy(deep=True)
    
    # Filtrage
    if Element_search:
        df = df[df['Element'] == Element_search]
    if Item_search:
        df = df[df['Item'] == Item_search]
    if Source_search:
        df = df[df['Source'] == Source_search]

    # Calcul de la somme
    somme = round(df["Emission_value"].sum(), 3)
    # Retourne la somme formatée
    return f' {somme} kt'
  
 
##################################################################
# Callback pour la carte
@app.callback(
    Output(component_id='carte', component_property='figure'),
    [Input(component_id='Element_filtre', component_property='value'),
     Input(component_id='Item_filtre', component_property='value'),
     Input(component_id='Source_filtre', component_property='value')]
)
def update_plot(Element_search, Item_search, Source_search):
    # Copie du dataframe
    df = df_transforme.copy(deep=True)
    
    # Filtrage
    if Element_search:
        df = df[df['Element'] == Element_search]
    if Item_search:
        df = df[df['Item'] == Item_search]
    if Source_search:
        df = df[df['Source'] == Source_search]
    
    # Carte choroplèthe
    fig = px.choropleth(df, locations='Area Code (ISO3)', title=f'Carte {Element_search}:',
                        color='Emission_value', animation_frame='Year', hover_name='Area',
                        color_continuous_scale="oranges")
 
    # Mise en page
    fig.update_layout(
        plot_bgcolor='rgba(0, 0, 0, 0)',  
        paper_bgcolor='rgba(0, 0, 0, 0)')
    return fig
##################################################################
# Callback pour le graphique linéaire
@app.callback(
    Output(component_id='Area_evolution', component_property='figure'),
    [Input(component_id='Element_filtre', component_property='value'),
     Input(component_id='Item_filtre', component_property='value'),
     Input(component_id='Source_filtre', component_property='value')]
)
def update_plot(Element_search, Item_search, Source_search):
    # Copie du dataframe
    df = df_transforme.copy(deep=True)
    
    # Filtrage
    if Element_search:
        df = df[df['Element'] == Element_search]
    if Item_search:
        df = df[df['Item'] == Item_search]
    if Source_search:
        df = df[df['Source'] == Source_search]
    
    # Données agrégées et graphique linéaire
    dataset = df.groupby('Year').agg({'Emission_value': 'sum'})
    fig = px.line(dataset, x=dataset.index, y='Emission_value', title=f'Tendances {Element_search}:')
    fig.update_layout(
        plot_bgcolor='rgba(0, 0, 0, 0)',  
        paper_bgcolor='rgba(0, 0, 0, 0)')
    return fig
##################################################################
# Callback pour le graphique à barres
@app.callback(
    Output(component_id='continent_bar', component_property='figure'),
    [Input(component_id='Element_filtre', component_property='value'),
     Input(component_id='Item_filtre', component_property='value'),
     Input(component_id='Source_filtre', component_property='value')]
)
def update_plot(Element_search, Item_search, Source_search):
    # Copie du dataframe
    df = df_transforme.copy(deep=True)
    
    # Filtrage
    if Element_search:
        df = df[df['Element'] == Element_search]
    if Item_search:
        df = df[df['Item'] == Item_search]
    if Source_search:
        df = df[df['Source'] == Source_search]
    
    # Données agrégées et graphique à barres
    dataset = df.groupby('continent').agg({'Emission_value': 'sum'})
    fig = px.bar(dataset, x=dataset.index, y='Emission_value', color='Emission_value', 
                 color_continuous_scale="oranges", title=f'{Element_search} par Continent:')
    fig.update_xaxes(title_text='')
    fig.update_layout(
        plot_bgcolor='rgba(0, 0, 0, 0)',  
        paper_bgcolor='rgba(0, 0, 0, 0)')
    return fig
##################################################################
# Callback pour le graphique sunburst
@app.callback(
    Output(component_id='Area_con', component_property='figure'),
    [Input(component_id='Element_filtre', component_property='value'),
     Input(component_id='Item_filtre', component_property='value'),
     Input(component_id='Source_filtre', component_property='value')]
)
def update_plot(Element_search, Item_search, Source_search):
    # Copie du dataframe
    df = df_transforme.copy(deep=True)
    
    # Filtrage
    if Element_search:
        df = df[df['Element'] == Element_search]
    if Item_search:
        df = df[df['Item'] == Item_search]
    if Source_search:
        df = df[df['Source'] == Source_search]
    
    # Données agrégées et graphique sunburst
    df2 = df.groupby(['continent', 'Area']).agg({'Emission_value': 'sum'}).reset_index()[['continent', 'Area', 'Emission_value']]
    fig = px.sunburst(df2, path=['continent', 'Area'], values='Emission_value',
                      title=f'{Element_search} par Pays/Continent:', color="continent")
    fig.update_layout(
        plot_bgcolor='rgba(0, 0, 0, 0)',
        paper_bgcolor='rgba(0, 0, 0, 0)')
    return fig
##################################################################
# Callback pour le graphique sunburst des systèmes agroalimentaires
@app.callback(
    Output(component_id='xxxxx', component_property='figure'),
    [Input(component_id='Element_filtre', component_property='value'),
    Input(component_id='Source_filtre', component_property='value')]
)
def update_plot(Element_search, Source_search):
    # Copie du dataframe
    df = df_transforme.copy(deep=True)
    
    # Filtrage
    if Element_search:
        df = df[df['Element'] == Element_search]
    if Source_search:
        df = df[df['Source'] == Source_search]
    
    # Filtrage pour les données agroalimentaires
    dataset = df[df['Item'].isin(liste_agro)]
    dataset = dataset.groupby('Item').agg({'Emission_value': 'sum'}).reset_index()[['Item', 'Emission_value']]
    fig = px.sunburst(dataset, path=['Item'], values='Emission_value',
               title="Émissions des systèmes agroalimentaires par composant")
    fig.update_layout(
        plot_bgcolor='rgba(0, 0, 0, 0)',
        paper_bgcolor='rgba(0, 0, 0, 0)')
    return fig

##################################################################

if __name__ == '__main__':
    app.run_server(debug=True, port=8199)

